In [1]:
import torch
from torch.utils.data import TensorDataset, Dataset
import torch.optim as optim
import torch.utils.data as data
from torchvision.transforms import transforms
from sklearn.metrics import f1_score
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from torchmetrics.classification import BinaryF1Score
from datetime import datetime

In [2]:
class BinaryCNN(nn.Module):
    """
    A Convolutional Neural Network (CNN) model for binary classification.

    Args:
    img_size (int): Size of input images.

    Attributes:
    features (nn.Sequential): Feature extraction part of the convolutional neural network.
    classifier (nn.Sequential): Fully connected classification part.
    """

    def __init__(self):
        super().__init__()
        
            
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.2),
            nn.Linear(64 * 32*32, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p = 0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.2),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        """        Args:
        x (torch.Tensor): Input data tensor.
        
        Returns:
        torch.Tensor: Output tensor of the model.
        """
        x = self.features(x)
        x = x.reshape(x.size(0), -1)
        x = self.classifier(x)
        
        return x

In [3]:
class CustomTensorDataset(Dataset):
    """
    A custom dataset class that works with tensors.

    Args:
    tensors (tuple): A tuple consisting of input data and target data tensors.
    transform (callable, optional): A function to apply data transformations.

    Attributes:
    tensors (tuple): A tuple consisting of input data and target data tensors.
    transform (callable): Data transformation function.
    size (int): Size of the dataset.
    """
    def __init__(self, tensors, transform=None):
        self.tensors = tensors
        self.transform = transform
        self.size = tensors[1].shape[0]


    def __getitem__(self, index):
        """
        Belirli bir dizindeki veri örneğini getirir.

        Args:
            index (int): Veri örneğinin dizin değeri.

        Returns:
            tuple: İlgili veri örneği ve hedef değeri.

        """
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        """
        Returns the size of the dataset.
        
        Returns:
        int: Size of the dataset.
        """

        return self.size

In [4]:
def visualize_loss(train_losses,val_losses,val_accuracy):
    """
    A function that visualizes training and validation loss along with validation accuracy.
    
    Args:
    train_losses (list): List of training losses.
    val_losses (list): List of validation losses.
    val_accuracy (list): List of validation accuracies.
    """

    fig, ax = plt.subplots(nrows=2, ncols=1,figsize=(14,10))
    ax[0].plot(range(0, len(train_losses) * 10, 10), train_losses, label='Training Loss')
    ax[0].plot(range(0, len(val_losses) * 10, 10), val_losses, label='Validation Loss')
    ax[0].set_xlabel('Steps')
    ax[0].set_ylabel('Loss')
    ax[0].title.set_text('Training and Validation Loss')
    ax[0].legend()

    ax[1].plot(range(0, len(train_losses) * 10, 10), val_accuracy, label='Validation Accuracy')
    ax[1].set_xlabel('Steps')
    ax[1].set_ylabel('Accuracy')
    ax[1].title.set_text('Validation Accuracy')
    plt.savefig('my_plot.pdf')
    plt.show()

In [5]:
def read_dataset(paths):
    """
    A function that reads images from given file paths.

    Args:
    paths (list): List of image file paths.

    Returns:
    torch.Tensor: Tensor of the read images.
    """
        
    images = []
    for path in paths:
        image = cv2.imread(path)
        images.append(image/255) #scaling
    images = np.array(images,dtype=np.float32)
    images = np.transpose(images, (0, 3, 1, 2)) # PyTorch library accepts inputs as (channel, height, width)

    images = torch.from_numpy(images)
    return images

In [6]:
def eval_model(val_loader,model,criterion,device):
    """
    A function to evaluate the model.
    
    Args:
    val_loader (torch.utils.data.DataLoader): Validation data loader.
    model (torch.nn.Module): Model to be evaluated.
    criterion (torch.nn.Module): Loss function.
    device (torch.device): Device used (CPU or GPU).
    
    Returns:
    tuple: Validation Loss, Accuracy, and F1 score.
    """

    model.eval()  
    val_loss = 0
    correct = 0
    total = 0
    f1 = 0
    f1_metric = BinaryF1Score().to(device)
    with torch.no_grad():
        for batch_idx, (paths, labels) in enumerate(val_loader):
            inputs = read_dataset(paths)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            val_loss += loss.item() * batch_size
            predicted = (outputs >= 0.5).squeeze().long()
            if (predicted == labels).sum().item() == 0:
                print(predicted,labels)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            f1 += f1_score(predicted.cpu().to('cpu'),labels,zero_division=1)
               
    accuracy = 100 * correct / total
    f1 = f1 / len(val_loader)
    val_loss = val_loss / len(val_loader)

    return val_loss, accuracy,f1

In [7]:
def load_model(path):
    """
    A function that loads the model and optimizer from the specified path.
    
    Args:
    path (str): Path to the file where the model and optimizer are saved.
    
    Returns:
    tuple: Loaded model and optimizer.
    """
    model = BinaryCNN().to(device)
    optimizer = optim.Adam(model.parameters())
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model,optimizer

In [8]:
db_path = 'db.csv'
test_files = ['Training_phase_2_046','Training_phase_2_047','Training_phase_2_048','Training_phase_2_049','Training_phase_2_050']

db = pd.read_csv(db_path)
# Delete the images designated as test files from the database.
db = db[~db.name.isin(test_files)]

# Seperating positive and negative examples
positive_sample = db[db.label == 1]
negative_sample = db[db.label == 0]
print(f"all data shape: {db.shape[0]} positive label number: {positive_sample.shape[0]}, negative label number: {negative_sample.shape[0]}")

# Creating a balanced dataset by sampling an equal number of negative and positive examples.
db = pd.concat([positive_sample, negative_sample.sample(n = positive_sample.shape[0])], ignore_index=True).sample(frac=1).reset_index(drop=True)

img_size = (256,256)


all data shape: 943596 positive label number: 286838, negative label number: 590882


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = BinaryCNN().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters())
f1_metric = BinaryF1Score().to(device)
train_losses = []
val_losses = []
val_accuracy = []

In [11]:
batch_size = 32

# Taking paths and labels of the images and creating a dataset.
X = db['img_path'].values
Y = db['label'].values
dataset = CustomTensorDataset(tensors=(X, Y))

# determining train and validation sizes.
train_size = int(0.95* len(dataset))
validation_size = len(dataset) - train_size

# Train and validation dataloaders are created according to the specified batch size.
train_dataset, val_dataset = data.random_split(dataset, [train_size, validation_size])
batch_size = 32
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
#Training Loop
for epoch in tqdm(range(5)):
  running_loss = 0.0

  for batch_idx, (paths, labels) in enumerate(train_loader):
    inputs = read_dataset(paths)
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
      
    outputs = model(inputs)

    loss = criterion(outputs, labels.float().unsqueeze(1))
    loss.backward()
    optimizer.step()

    # Loss is calculated for each batch and accumulated cumulatively as we progress.
    running_loss += loss.item() * batch_size
    
    
    if batch_idx % 300 == 299:
        model.eval()    
        val_loss = 0
        correct = 0
        total = 0
        f1 = 0
        running_loss = running_loss / 300
        with torch.no_grad():
            for batch_idx, (paths, labels) in enumerate(val_loader):
                inputs = read_dataset(paths)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels.float().unsqueeze(1))
                val_loss += loss.item() * batch_size
                predicted = (outputs >= 0.5).squeeze().long()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                f1 += f1_score(labels.cpu(),predicted.to('cpu'),zero_division=1)
                   
        accuracy = 100 * correct / total
        f1 = f1 / len(val_loader)
        val_loss = val_loss / len(val_loader)
        
        train_losses.append(running_loss)
        val_losses.append(val_loss)
        print(f"train_loss:{running_loss}, val_loss:{val_loss}, accuracy:{accuracy}, f1:{f1} ")
        val_accuracy.append(accuracy)
        running_loss= 0

  # The model and optimizer are saved at each epoch based on the timestamp when they are recorded.
  checkpoint = {
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict()
  }
  now = datetime.now()
  current_time = now.strftime("_%H_%M")
  MODEL_PATH = f"model/model_checkpoint{current_time}.pth"
  torch.save(checkpoint, MODEL_PATH)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

train_loss:10.999216832319895, val_loss:10.517222729016307, accuracy:86.25017431320597, f1:0.8713336244342439 
train_loss:16.67396251996358, val_loss:14.201749363073139, accuracy:83.10905034165388, f1:0.8107450034703871 
train_loss:12.20600115219752, val_loss:11.350242028666978, accuracy:84.15144331334542, f1:0.8178328503989138 
train_loss:9.761590174039204, val_loss:7.672654734008687, accuracy:90.81718030958025, f1:0.9045473012319235 
train_loss:8.608665653069814, val_loss:8.285102360756236, accuracy:89.46102356714545, f1:0.8903230110560493 
train_loss:8.386354035933813, val_loss:6.779514581795653, accuracy:92.0548040719565, f1:0.9170821384411668 
train_loss:7.868975687821706, val_loss:8.149372725050851, accuracy:89.82010877144053, f1:0.8949704977328484 
train_loss:8.2660282989343, val_loss:6.8209587549551935, accuracy:91.44122158694742, f1:0.9079294086931852 
train_loss:7.773365598519643, val_loss:9.434138224675106, accuracy:86.5465067633524, f1:0.8432514709449269 
train_loss:7.21193

 20%|███████████████▏                                                            | 1/5 [3:03:49<12:15:16, 11029.14s/it]

train_loss:3.239495050062736, val_loss:3.211637331523757, accuracy:96.39171663645237, f1:0.9626576692631064 
train_loss:3.412564188738664, val_loss:3.1738028385434527, accuracy:96.24180727931949, f1:0.961392768409703 
train_loss:3.2715665039916835, val_loss:3.164584036305995, accuracy:96.47190071119788, f1:0.9636212594718425 
train_loss:3.5857661642630894, val_loss:3.2858786161993128, accuracy:96.16859573281272, f1:0.9602425755294552 
train_loss:2.8651709021627902, val_loss:5.717835865806842, accuracy:94.12564495886208, f1:0.9417459931965144 
train_loss:3.280411051760117, val_loss:3.814628752228807, accuracy:95.88620833914378, f1:0.9562044303473363 
train_loss:3.5330753080546855, val_loss:3.9523204227072606, accuracy:95.77813415144331, f1:0.9548999132456434 
train_loss:3.5303494815031686, val_loss:3.7514809464398304, accuracy:95.78510667968206, f1:0.9572339440289327 
train_loss:3.6796632164220013, val_loss:3.214546908768789, accuracy:96.31501882582624, f1:0.960740845446568 
train_loss:

 40%|██████████████████████████████▊                                              | 2/5 [5:55:37<8:50:14, 10604.99s/it]

train_loss:2.1520681726187467, val_loss:2.3716070130716598, accuracy:97.46548598521824, f1:0.9732162933155493 
train_loss:2.1561875702937443, val_loss:2.414634548181641, accuracy:97.45502719286013, f1:0.9731196535769026 
train_loss:2.319766070395708, val_loss:2.7220597668709763, accuracy:97.20401617626551, f1:0.9711442661113636 
train_loss:2.3857458981623254, val_loss:3.161376449940861, accuracy:96.41612048528796, f1:0.9619315729058624 
train_loss:2.4066338406006493, val_loss:2.629157807216995, accuracy:97.21447496862362, f1:0.970609304576997 
train_loss:2.2694017195453244, val_loss:2.8865638787763706, accuracy:96.92511504671594, f1:0.9672684321531977 
train_loss:2.2698156623045604, val_loss:3.1458992068982843, accuracy:96.74382931250872, f1:0.9655592507462553 
train_loss:2.339317378997803, val_loss:2.4983199645028997, accuracy:97.27722772277228, f1:0.9714311991067103 
train_loss:2.3588885319481294, val_loss:2.774061905434033, accuracy:97.02273044205829, f1:0.9687814732251757 
train_lo

 60%|██████████████████████████████████████████████▏                              | 3/5 [8:46:33<5:48:11, 10445.64s/it]

train_loss:1.7450651265432437, val_loss:3.3266808277882696, accuracy:96.88676614140287, f1:0.9676545042236853 
train_loss:1.5546559400421878, val_loss:2.8423563786922488, accuracy:97.09245572444568, f1:0.9696213787515718 
train_loss:1.6034556831419469, val_loss:2.5824629022524905, accuracy:97.50034862641193, f1:0.9738755879694864 
train_loss:1.6660728384678563, val_loss:3.1813461503871574, accuracy:96.43355180588482, f1:0.9637217065473168 
train_loss:1.8325281751155853, val_loss:2.3149666005294924, accuracy:97.57007390879933, f1:0.9746012544209167 
train_loss:1.7183557519999644, val_loss:2.566587003174268, accuracy:97.4236508157858, f1:0.9730874008030357 
train_loss:2.130656378803154, val_loss:3.3819833698048107, accuracy:96.62878259656952, f1:0.964806175041971 
train_loss:2.625242778112491, val_loss:2.9916036217623994, accuracy:96.78566448194115, f1:0.9664182336771686 
train_loss:1.8556884260227282, val_loss:2.7880093719137453, accuracy:97.29117277924976, f1:0.9714749579467599 
train_

 80%|████████████████████████████████████████████████████████████▊               | 4/5 [11:49:17<2:57:30, 10650.33s/it]

train_loss:1.3873573241072397, val_loss:3.086884297704617, accuracy:96.91116999023846, f1:0.9675983396511486 
train_loss:1.181206127529343, val_loss:3.5881048840057383, accuracy:96.97740900850648, f1:0.9686939146417789 


In [ ]:
visualize_loss(train_losses,val_losses,val_accuracy)